# Fine-tuning Phi-2 for Insurance Domain

This notebook demonstrates fine-tuning Microsoft's Phi-2 model using **LoRA (Low-Rank Adaptation)** for insurance claims processing tasks. We'll train the model on instruction-response pairs to generate domain-specific explanations for claim denials and appeals.

## Workflow Overview
1. **Data Preparation**: Create and format insurance claims dataset
2. **Model Setup**: Load Phi-2 and configure LoRA adapter
3. **Training**: Fine-tune the model with optimized settings
4. **Inference**: Generate responses for new queries

In [ ]:
from datasets import Dataset

# Define training data: insurance claim instruction-response pairs
data = [
    {
        "instruction": "Explain why CPT 70553 was denied.",
        "response": "Authorization denied because neurological deficit criteria required under Policy 4.12 were not documented."
    },
    {
        "instruction": "Write a short appeal for denied MRI brain claim.",
        "response": "We respectfully request reconsideration as the patient presented worsening neurological symptoms that meet medical necessity criteria."
    },
    {
        "instruction": "Explain denial for incomplete prior authorization.",
        "response": "Authorization denied due to missing clinical documentation supporting medical necessity."
    }
]

# Convert to Hugging Face Dataset format for easy processing
dataset = Dataset.from_list(data)
print(f"Dataset created with {len(dataset)} examples")
dataset

## 1. Data Preparation

### Create Insurance Claims Dataset
We'll create a small dataset of instruction-response pairs focused on insurance claim denials and appeals. Each example contains:
- **instruction**: The user's query about insurance matters
- **response**: The model's expected response

This dataset serves as the training data for fine-tuning Phi-2.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Specify the model to fine-tune
model_name = "microsoft/phi-2"

# Load the tokenizer - converts text to token IDs
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the pre-trained Phi-2 model - 2.7B parameter causal language model
model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"✅ Loaded model: {model_name}")

## 2. Model Initialization

### Load Phi-2 Model and Tokenizer
We load Microsoft's Phi-2 model (2.7B parameters) and its tokenizer. This lightweight model is ideal for fine-tuning on consumer hardware while maintaining strong performance on domain-specific tasks.

In [ ]:
def format_example(example):
    """
    Format a single example into the instruction-response prompt structure.
    
    Args:
        example: Dict with 'instruction' and 'response' keys
        
    Returns:
        Dict with 'text' key containing the formatted prompt
    """
    return {
        "text": f"""### Instruction:
{example['instruction']}

### Response:
{example['response']}"""
    }

# Apply formatting to all dataset examples
formatted_dataset = dataset.map(format_example)
print(f"Dataset formatted with {len(formatted_dataset)} examples")
formatted_dataset

## 3. Dataset Formatting

### Format Data for Language Modeling
Convert each instruction-response pair into a formatted text string structured as:
```
### Instruction:
{instruction}

### Response:
{response}
```

This format helps the model understand the task structure during training.

In [ ]:
from peft import LoraConfig, get_peft_model

# Configure LoRA (Low-Rank Adaptation) parameters
lora_config = LoraConfig(
    r=8,                                              # Rank of the low-rank matrices (lower = fewer parameters)
    lora_alpha=16,                                    # Scaling factor for LoRA weights
    target_modules=["Wqkv", "fc1", "fc2"],            # Phi-2 transformer layers to apply LoRA to
    lora_dropout=0.05,                                # Dropout for LoRA layers
    bias="none",                                       # No bias in LoRA modules
    task_type="CAUSAL_LM"                             # Task type: Causal Language Modeling
)

# Wrap the base model with LoRA adapters
peft_model = get_peft_model(model, lora_config)

# Display trainable parameters
print("🔧 LoRA Configuration Applied:")
peft_model.print_trainable_parameters()

## 4. LoRA Configuration

### Apply Low-Rank Adaptation
**LoRA (Low-Rank Adaptation)** is a parameter-efficient fine-tuning technique that:
- Adds trainable low-rank matrices to attention layers
- Reduces the number of trainable parameters from millions to thousands
- Enables fine-tuning on consumer hardware without storing full model weights

We configure LoRA for Phi-2's specific architecture targeting the attention and feed-forward layers.

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# ===== STEP 1: TOKENIZATION SETUP =====
# Set pad token for tokenizer (required for batching)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define tokenization function
def tokenize_function(examples):
    """
    Tokenize text examples for language modeling.
    
    Args:
        examples: Dict with 'text' key containing raw text
        
    Returns:
        Tokenized inputs with token IDs and attention masks
    """
    return tokenizer(examples["text"], truncation=True, max_length=512)

# Tokenize the entire dataset
tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)
print(f"✅ Dataset tokenized: {len(tokenized_dataset)} examples")

# ===== STEP 2: DATA COLLATOR =====
# Data collator for causal language modeling (next token prediction)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # False for causal LM (predict next token), True for masked LM
)

# ===== STEP 3: TRAINING ARGUMENTS =====
training_args = TrainingArguments(
    output_dir="./phi2-insurance",           # Where to save model checkpoints
    per_device_train_batch_size=1,            # Batch size (1 for consumer hardware)
    num_train_epochs=3,                       # Number of training epochs
    logging_steps=1,                          # Log metrics every N steps
    save_strategy="no",                       # Don't save intermediate checkpoints
    fp16=False,                               # Disable mixed precision (avoid precision issues)
    learning_rate=2e-4,                       # Learning rate for optimization
    weight_decay=0.01,                        # L2 regularization
    logging_dir="./logs",                      # Directory for TensorBoard logs
    evaluation_strategy="epoch"                # Evaluate at the end of each epoch
)

# ===== STEP 4: INITIALIZE TRAINER =====
trainer = Trainer(
    model=peft_model,                         # The PEFT-wrapped model with LoRA adapters
    args=training_args,                       # Training configuration
    train_dataset=tokenized_dataset,          # Training data
    data_collator=data_collator,              # Prepare batches for training
    tokenizer=tokenizer                       # Tokenizer for special token handling
)

# ===== STEP 5: TRAIN THE MODEL =====
print("\n🚀 Starting fine-tuning on insurance claims data...")
trainer.train()
print("✅ Fine-tuning complete! Model ready for inference.")

 11%|█         | 1/9 [01:16<10:14, 76.81s/it]

{'loss': 10.3174, 'learning_rate': 0.00017777777777777779, 'epoch': 0.33}


 22%|██▏       | 2/9 [02:08<07:14, 62.02s/it]

{'loss': 10.7894, 'learning_rate': 0.00015555555555555556, 'epoch': 0.67}


 33%|███▎      | 3/9 [02:09<03:24, 34.08s/it]

...

{'loss': 7.5497, 'learning_rate': 4.4444444444444447e-05, 'epoch': 2.33}


 89%|████████▉ | 8/9 [02:12<00:04,  4.52s/it]

{'loss': 8.3521, 'learning_rate': 2.2222222222222223e-05, 'epoch': 2.67}


100%|██████████| 9/9 [02:13<00:00, 14.80s/it]

{'loss': 7.7689, 'learning_rate': 0.0, 'epoch': 3.0}
{'train_runtime': 133.2259, 'train_samples_per_second': 0.068, 'train_steps_per_second': 0.068, 'train_loss': 9.038271639082167, 'epoch': 3.0}
✅ Fine-tuning complete! Model ready for inference.


## 5. Training Setup & Execution

### Tokenization and Training Configuration
This step:
1. **Tokenizes** the formatted text into token IDs the model understands
2. **Configures training arguments** (learning rate, batch size, epochs, etc.)
3. **Initializes the Trainer** with the PEFT model, dataset, and training config
4. **Trains** the model on the insurance claims data

**Note**: We use a batch size of 1 for memory efficiency on consumer hardware.

In [ ]:
def generate(text):
    """
    Generate text using the fine-tuned Phi-2 model.
    
    Args:
        text: Input prompt string
        
    Returns:
        Generated text continuation
    """
    # Detect the appropriate device for inference
    if torch.backends.mps.is_available():
        device = "mps"  # Apple Metal Performance Shaders
    elif torch.cuda.is_available():
        device = "cuda"  # NVIDIA CUDA
    else:
        device = "cpu"  # CPU fallback
    
    # Tokenize the input prompt and move to the correct device
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    # Generate text with the model in evaluation mode
    peft_model.eval()
    with torch.no_grad():  # Disable gradient computation for faster inference
        output = peft_model.generate(
            **inputs,
            max_length=200,      # Maximum output length
            do_sample=False      # Greedy decoding (deterministic)
        )
    
    # Decode token IDs back to readable text
    return tokenizer.decode(output[0], skip_special_tokens=True)

# ===== EXAMPLE INFERENCE =====
# Create a test prompt following the instruction-response format
prompt = """### Instruction:
Explain denial for missing clinical documentation.

### Response:
"""

# Generate response from the fine-tuned model
print("🔄 Generating response from fine-tuned Phi-2 model...")
print("\n" + "="*60)
print(generate(prompt))
print("="*60)

## 6. Inference & Text Generation

### Generate Responses with Fine-tuned Model
The `generate()` function:
1. **Detects hardware**: Uses MPS (Mac), CUDA (GPU), or CPU
2. **Tokenizes input**: Converts the prompt to token IDs
3. **Generates output**: Uses the fine-tuned model to generate responses
4. **Decodes output**: Converts token IDs back to human-readable text

The model is evaluated in `eval()` mode and uses `torch.no_grad()` for memory efficiency during inference.

In [ ]:
## Summary & Next Steps

### What We Accomplished
✅ Created an insurance claims dataset with instruction-response pairs  
✅ Loaded the Phi-2 model and applied LoRA adapters  
✅ Formatted and tokenized the data for training  
✅ Fine-tuned the model on domain-specific insurance data  
✅ Demonstrated inference with the trained model  

### Key Concepts Covered
- **Parameter-Efficient Fine-tuning**: LoRA reduces trainable parameters by 99%+
- **Domain Adaptation**: Customizing a general-purpose model for insurance tasks
- **Hardware Optimization**: Multi-device support (MPS/CUDA/CPU)
- **Instruction-Response Formatting**: Structured prompts for better model behavior

### Next Steps to Explore
1. **Expand Dataset**: Collect more insurance claim examples for better generalization
2. **Evaluate Performance**: Test on held-out validation data and calculate metrics
3. **Fine-tune Hyperparameters**: Experiment with learning rate, rank, dropout
4. **Save & Load Model**: Persist LoRA weights for production use
5. **Deploy**: Create an API endpoint for real-time inference

### Resources
- [Phi-2 Model Card](https://huggingface.co/microsoft/phi-2)
- [PEFT Library Documentation](https://github.com/huggingface/peft)
- [Hugging Face Transformers](https://huggingface.co/docs/transformers/)